In [3]:
import os
import tensorflow as tf
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM

In [6]:
model = Sequential()

In [8]:
print(model.summary)

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x00000231B176D040>>


In [9]:
model = Sequential()
model.add(LSTM(3, input_shape=(2, 10)))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3)                 168       
                                                                 
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model = Sequential()
model.add(LSTM(3, batch_input_shape=(8, 2, 10)))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (8, 3)                    168       
                                                                 
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
samples = ['Was it a cat I saw', 'To be or not to be']
token_index = {} 
for sam in samples:
    for word in sam.split():
        if word not in token_index:
            #print(word)
            token_index[word] = len(token_index)
print(token_index)

{'Was': 0, 'it': 1, 'a': 2, 'cat': 3, 'I': 4, 'saw': 5, 'To': 6, 'be': 7, 'or': 8, 'not': 9, 'to': 10}


In [12]:
print()
tokenizer = Tokenizer(num_words=10)  # 토큰화 객체 생성. 빈도가 높은 10개까지의 단어 만으로 벡터 생성
tokenizer.fit_on_texts(samples)  # 대상은 list type
token_seq = tokenizer.texts_to_sequences(samples)
print(token_seq)   #


[[3, 4, 5, 6, 7, 8], [1, 2, 9, 1, 2]]


In [13]:
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
# token으로 matrix화
token_mat = tokenizer.texts_to_matrix(samples, mode="binary") # 'count','tfidf','freq'
print(token_mat)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[[0. 0. 0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 1.]]


In [14]:
word_index = tokenizer.word_index  # 각 단어에 매겨진 인덱스 값 출력
print('Found %s unique tokens'%(len(word_index)))  # Found 9 unique tokens
print(word_index)

Found 10 unique tokens
{'to': 1, 'be': 2, 'was': 3, 'it': 4, 'a': 5, 'cat': 6, 'i': 7, 'saw': 8, 'or': 9, 'not': 10}


In [15]:
print(tokenizer.word_counts) 

OrderedDict([('was', 1), ('it', 1), ('a', 1), ('cat', 1), ('i', 1), ('saw', 1), ('to', 2), ('be', 2), ('or', 1), ('not', 1)])


In [17]:
from tensorflow.keras.utils import to_categorical
token_seq = to_categorical(token_seq[0], num_classes=10)
print(token_seq)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [18]:
import numpy as np    # 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정.
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]
classes = np.array([1,1,1,1,1,0,0,0,0,0])

In [19]:
token = Tokenizer()    # 토큰화
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [20]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)

# 패딩 : 서로 다른 길이의 데이터를 4로 맞추어 준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_x = pad_sequences(x, 4)  
print("\n패딩 결과:\n", padded_x)

print("\n딥러닝 모델 시작:")
word_size = len(token.word_index) + 1  # 임베딩에 입력될 단어의 수를 지정

# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

model = Sequential()
# Embedding(가능 토큰 수, 임베딩 차원, input_length=시퀀스 수)
# 정수를 입력으로 받아 내부 딕셔너리에서 이 정수에 연관된 벡터를 차아 반환함
model.add(Embedding(word_size, 8, input_length=4)) 
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]

패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]

딥러닝 모델 시작:
Epoch 1/20
1/1 [==============================] - 1s 634ms/step - loss: 0.7082 - accuracy: 0.1000
Epoch 2/20
1/1 [==============================] - 0s 7ms/step - loss: 0.7057 - accuracy: 0.1000
Epoch 3/20
1/1 [==============================] - 0s 6ms/step - loss: 0.7032 - accuracy: 0.2000
Epoch 4/20
1/1 [==============================] - 0s 6ms/step - loss: 0.7008 - accuracy: 0.3000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6983 - accuracy: 0.3000
Epoch 6/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6959 - accuracy: 0.4000
Epoch 7/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.5000
Epoch 8/20
1/1 [============================